# groupby 메서드로 평균값 구하기

In [1]:
import pandas as pd 
df = pd.read_csv('../data/gapminder.tsv', sep='\t')

In [32]:
avg_life_exp_by_year = df.groupby('year').lifeExp.mean() 
print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


# 분할-반영-결합 과정 살펴보기

In [3]:
years = df.year.unique() 
print(years)

[1952 1957 1962 1967 1972 1977 1982 1987 1992 1997 2002 2007]


In [4]:
y1952 = df.loc[df.year == 1952, :] 
print(y1952.head())

        country continent  year  lifeExp       pop    gdpPercap
0   Afghanistan      Asia  1952   28.801   8425333   779.445314
12      Albania    Europe  1952   55.230   1282697  1601.056136
24      Algeria    Africa  1952   43.077   9279525  2449.008185
36       Angola    Africa  1952   30.015   4232095  3520.610273
48    Argentina  Americas  1952   62.485  17876956  5911.315053


In [5]:
y1952_mean = y1952.lifeExp.mean() 
print(y1952_mean)

49.057619718309866


In [6]:
y1957 = df.loc[df.year == 1957, :] 
y1957_mean = y1957.lifeExp.mean( )
print(y1957_mean)

y1962 = df.loc[df.year == 1962, :] 
y1962_mean = y1962.lifeExp.mean( )
print(y1962_mean)

y2007 = df.loc[df.year == 2007, :] 
y2007_mean = y2007.lifeExp.mean( )
print(y2007_mean)

51.50740112676056
53.609249014084504
67.00742253521126


In [7]:
df2 = pd.DataFrame({"year":[1952, 1957, 1962, 2007], 
                    "":[y1952_mean, y1957_mean,y1962_mean,y2007_mean]}) 
print(df2)

   year           
0  1952  49.057620
1  1957  51.507401
2  1962  53.609249
3  2007  67.007423


# 평균값을 구하는 사용자 함수와 groupby 메서드

In [8]:
def my_mean(values):
    n = len(values)
    
    sum = 0 
    for value in values:
        sum += value
    
    return sum / n

In [9]:
agg_my_mean = df.groupby('year').lifeExp.agg(my_mean) 
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


# 두 개의 인잣값을 받아 처리하는 사용자 함수와 groupby 메서드

In [10]:
def my_mean_diff(values, diff_value):
    n = len(values) 
    sum = 0 
    for value in values:
        sum += value 
    mean = sum / n 
    return mean - diff_value

In [11]:
global_mean = df.lifeExp.mean() 
print(global_mean)

59.474439366197174


In [12]:
agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, diff_value=global_mean) 
print(agg_mean_diff)

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


# 집계 메서드를 리스트, 딕셔너리에 담아 전달하기

In [33]:
import numpy as np
gdf = df.groupby('year').lifeExp.agg(['count', 'mean','std']) 
print(gdf)

      count       mean        std
year                             
1952    142  49.057620  12.225956
1957    142  51.507401  12.231286
1962    142  53.609249  12.097245
1967    142  55.678290  11.718858
1972    142  57.647386  11.381953
1977    142  59.570157  11.227229
1982    142  61.533197  10.770618
1987    142  63.212613  10.556285
1992    142  64.160338  11.227380
1997    142  65.014676  11.559439
2002    142  65.694923  12.279823
2007    142  67.007423  12.073021


In [34]:
gdf_dict = df.groupby('year').agg({'lifeExp': 'mean', 'pop': 'median', 'gdpPercap': 'median'}) 
print(gdf_dict)

        lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334740
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
1992  64.160338   8688686.5  4386.085502
1997  65.014676   9735063.5  4781.825478
2002  65.694923  10372918.5  5319.804524
2007  67.007423  10517531.0  6124.371108


# 표준점수 계산하기

In [35]:
def my_zscore(x):
    return (x - x.mean()) / x.std()

In [36]:
transform_z = df.groupby('year').lifeExp.transform(my_zscore)

print(transform_z.head())

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64


In [37]:
print(df.shape)

(1704, 6)


In [38]:
print(transform_z.shape)

(1704,)


# 누락값을 평균값으로 처리하기

In [39]:
import seaborn as sns 
import numpy as np

np.random.seed(42)
tips_10 = sns.load_dataset('tips').sample(10)
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN

print(tips_10)

AttributeError: `np.NaN` was removed in the NumPy 2.0 release. Use `np.nan` instead.

In [1]:
count_sex = tips_10.groupby('sex').count() 
print(count_sex)

NameError: name 'tips_10' is not defined

In [ ]:
def fill_na_mean(x):
    avg = x.mean() 
    return x.fillna(avg)

In [ ]:
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean
print(tips_10)

# 데이터 필터링 사용하기 ─ filter 메서드

In [20]:
tips = sns.load_dataset('tips')

print(tips.shape)

(244, 7)


In [21]:
print(tips['size'].value_counts())

size
2    156
3     38
4     37
5      5
1      4
6      4
Name: count, dtype: int64


In [22]:
tips_filtered = tips.\
    groupby('size').\
    filter(lambda x: x['size'].count() >= 30)

In [23]:
print(tips_filtered.shape)

(231, 7)


In [24]:
print(tips_filtered['size'].value_counts())

size
2    156
3     38
4     37
Name: count, dtype: int64


# 그룹 오브젝트 저장하여 살펴보기

In [25]:
tips_10 = sns.load_dataset('tips').sample(10, random_state=42) 
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153       24.55  2.00    Male     No   Sun  Dinner     4
211       25.89  5.16    Male    Yes   Sat  Dinner     4
198       13.00  2.00  Female    Yes  Thur   Lunch     2
176       17.89  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [26]:
grouped = tips_10.groupby('sex')
print(grouped)

C:\Users\윤성애\AppData\Local\Temp\ipykernel_21156\1549211312.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = tips_10.groupby('sex')


In [27]:
print(grouped.groups)

{'Male': [24, 6, 153, 211, 176, 192, 9], 'Female': [198, 124, 101]}


# 그룹 오브젝트의 평균 구하기

In [28]:
avgs = grouped.mean() 
print(avgs)

TypeError: category dtype does not support aggregation 'mean'

In [2]:
print(tips_10.columns)

NameError: name 'tips_10' is not defined

# 그룹 오브젝트에서 데이터 추출하고 반복하기

In [3]:
female = grouped.get_group('Female') 
print(female)

NameError: name 'grouped' is not defined

In [4]:
for sex_group in grouped:
    print(sex_group)

NameError: name 'grouped' is not defined

In [5]:
for sex_group in grouped:
    print('the type is: {}\n'.format(type(sex_group)))
    print('the length is: {}\n'.format(len(sex_group)))

    first_element = sex_group[0] 
    print('the first element is: {}\n'.format(first_element))
    print('it has a type of: {}\n'.format(type(sex_group[0])))

    second_element = sex_group[1] 
    print('the second element is:\n{}\n'.format(second_element))
    print('it has a type of: {}\n'.format(type(second_element)))

    print('what we have:') 
    print(sex_group)

    break

NameError: name 'grouped' is not defined

# 그룹 오브젝트 계산하고 살펴보기

In [30]:
bill_sex_time = tips_10.groupby(['sex', 'time'])
group_avg = bill_sex_time.mean() 

print(group_avg)

C:\Users\윤성애\AppData\Local\Temp\ipykernel_21156\2089557308.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bill_sex_time = tips_10.groupby(['sex', 'time'])


TypeError: category dtype does not support aggregation 'mean'

In [31]:
print(type(group_avg))

NameError: name 'group_avg' is not defined

In [ ]:
print(group_avg.columns)

In [ ]:
print(group_avg.index)

In [ ]:
group_method = tips_10.groupby(['sex', 'time']).mean().reset_index() 
print(group_method)

In [ ]:
group_param = tips_10.groupby(['sex', 'time'], as_index=False).mean( ) 
print(group_param)